In [2]:
import re
import json

class STAParser:
    def __init__(self, sta_report: str):
        self.report = sta_report

    def parse(self):
        paths = []
        blocks = self.report.strip().split("Startpoint:")
        for block in blocks[1:]:
            path = self._parse_block(block)
            if path:
                paths.append(path)
        return {"paths": paths}

    def _parse_block(self, block: str):
        lines = block.strip().splitlines()
        startpoint = lines[0].strip()
        endpoint = ""
        clock = ""
        path_type = ""
        data_arrival = None
        data_required = None
        slack = None
        status = None
        logic_chain = []

        # Extract metadata
        for line in lines:
            if line.startswith("Endpoint:"):
                endpoint = line.replace("Endpoint:", "").strip()
            elif line.startswith("Path Group:"):
                clock = line.replace("Path Group:", "").strip()
            elif line.startswith("Path Type:"):
                path_type = line.replace("Path Type:", "").strip()
            elif "data arrival time" in line and data_arrival is None:
                try:
                    data_arrival = float(line.split()[0])
                except:
                    pass
            elif "data required time" in line and data_required is None:
                try:
                    data_required = float(line.split()[0])
                except:
                    pass
            elif "slack" in line:
                parts = line.split()
                slack = float(parts[0])
                status = "VIOLATED" if "VIOLATED" in line else "MET"
            elif re.search(r"\s+[0-9]", line) and ("v " in line or "^ " in line):
                # Logic chain row
                parts = line.split()
                try:
                    delay = float(parts[0])
                except:
                    delay = 0.0
                description = " ".join(parts[2:])
                logic_chain.append({"cell": description, "delay": delay})

        # Compact JSON output
        if status == "VIOLATED":
            return {
                "startpoint": startpoint,
                "endpoint": endpoint,
                "clock": clock,
                "path_type": path_type,
                "data_arrival_time": data_arrival,
                "data_required_time": data_required,
                "slack": slack,
                "status": status,
                "logic_chain": logic_chain
            }
        else:  # summary mode
            return {
                "startpoint": startpoint,
                "endpoint": endpoint,
                "clock": clock,
                "path_type": path_type,
                "data_arrival_time": data_arrival,
                "data_required_time": data_required,
                "slack": slack,
                "status": status,
                "logic_chain": [x["cell"] for x in logic_chain]
            }


if __name__ == "__main__":
    with open(r"D:\Code\timing-violaton-debugger\app\data\timing_report.txt", "r") as f:
        report = f.read()

    parser = STAParser(report)
    output = parser.parse()

    with open("sta_output.json", "w") as f:
        json.dump(output, f, indent=2)


In [3]:
import os
import json
from dotenv import load_dotenv

from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Load API key from .env
load_dotenv()
if not os.getenv("GROQ_API_KEY"):
    raise ValueError("Missing GROQ_API_KEY in .env")

# Init Groq model (can swap model easily)
model = init_chat_model(
    "llama-3.3-70b-versatile",
    model_provider="groq",
    temperature=0.2  # lower temp for deterministic outputs
)


In [4]:
with open("sta_output.json", "r") as f:
    data = json.load(f)

paths = data.get("paths", [])  # our optimized Module-2 output
print(f"Loaded {len(paths)} timing paths.")


Loaded 2 timing paths.


In [5]:
# Few-shot example (curly braces doubled)
few_shot_examples = """
### EXAMPLE
Input:
{{
  "startpoint": "U1/Q",
  "endpoint": "U5/D",
  "slack": -0.85,
  "delay": 5.2,
  "clock": "clk_main"
}}

Output:
{{
  "root_cause": "Path delay exceeds the clock period, causing setup violation.",
  "suggestions": [
    "Insert a pipeline register to break the path.",
    "Optimize combinational logic to reduce delay.",
    "Replace slow gates with faster cells."
  ]
}}
"""

prompt_violation_debugger = PromptTemplate.from_template(
    f"""
You are a GenAI-powered timing violation debugger.
You will receive one STA path at a time in JSON format.

### Input Path JSON:
{{path_json}}

### INSTRUCTION:
1. If status = "VIOLATED":
   * Identify the root cause based on path_type and logic_chain.
   * Suggest an optimal fix (1–2 sentences).
2. If status = "MET":
   * root_cause = null
   * suggested_fix = null

### OUTPUT:
Return ONLY valid JSON with these keys:
"startpoint", "endpoint", "path_type", "status", "root_cause", "suggested_fix".

{few_shot_examples}
"""
)

# JSON parser
json_parser = JsonOutputParser()


In [6]:
results = []

for i, p in enumerate(paths, start=1):
    # Chain: prompt + model
    chain = prompt_violation_debugger | model
    
    # Invoke LLM
    res = chain.invoke({"path_json": json.dumps(p, indent=2)})
    
    # Parse output
    parsed_res = json_parser.parse(res.content)
    results.append(parsed_res)

print("Sample result:")
print(json.dumps(results[:1], indent=2))


Sample result:
[
  {
    "startpoint": "in1 (input port clocked by clk)",
    "endpoint": "r1 (rising edge-triggered flip-flop clocked by clk)",
    "path_type": "min",
    "status": "VIOLATED",
    "root_cause": "The minimum path delay is not meeting the required timing, likely due to insufficient delay in the logic chain or issues with clock synchronization.",
    "suggested_fix": "Review the logic chain for opportunities to insert delays or optimize the combinational logic to improve timing, and verify clock tree synchronization."
  }
]


In [7]:
print(json.dumps(results, indent=2))

[
  {
    "startpoint": "in1 (input port clocked by clk)",
    "endpoint": "r1 (rising edge-triggered flip-flop clocked by clk)",
    "path_type": "min",
    "status": "VIOLATED",
    "root_cause": "The minimum path delay is not meeting the required timing, likely due to insufficient delay in the logic chain or issues with clock synchronization.",
    "suggested_fix": "Review the logic chain for opportunities to insert delays or optimize the combinational logic to improve timing, and verify clock tree synchronization."
  },
  {
    "startpoint": "r2 (rising edge-triggered flip-flop clocked by clk)",
    "endpoint": "r3 (rising edge-triggered flip-flop clocked by clk)",
    "path_type": "max",
    "status": "MET",
    "root_cause": null,
    "suggested_fix": null
  }
]


In [8]:
output = {"analysis": results}

with open("llm_analysis.json", "w") as f:
    json.dump(output, f, indent=2)

print("Saved -> llm_analysis.json")


Saved -> llm_analysis.json
